In [5]:
import json_lines
import numpy as np
import html2text
import re
import pickle
import requests
import json
from pyquery import PyQuery as pq


In [2]:
# Load data
with open('top_articles_200k.jl', 'rb') as f:
    data = []
    for item in json_lines.reader(f):
        data.append(item)

In [3]:
# get index of triip
tripInsideIndex = []
tripOutsideIndex = []
for i,d in enumerate(data):
    if (d['site_category'] == '國內旅遊'):
        tripInsideIndex.append(i)
    if (d['site_category'] == '國外旅遊'):
        tripOutsideIndex.append(i)
data = np.array(data)

print(len(tripInsideIndex))
print(len(tripOutsideIndex))

34559
49996


In [4]:
def get_text_forTrip(data,tripInsideIndex,tripOutsideIndex):
    tripInside = data[tripInsideIndex]
    tripOutside = data[tripOutsideIndex]
    err_in = []
    err_out = []
    for i,trip_in in enumerate(tripInside):
        html = trip_in['body']
        
        try:
            if (type(html) != list):
                tripInside[i]['fix'] =  0
                text = html2text.html2text(html)
                text_pre = []
                for sentence in re.split('\[]|\n',text):
                    if (sentence != ''):
                        if('jpg' in sentence or 'gif' in sentence):
                            continue
                        else:
                            text_pre .append(sentence)
                tripInside[i]['body'] =  text_pre
                tripInside[i]['fix'] =  1
        except:
            print('Error',i)
            err_in.append(i)
            
    for i,trip_out in enumerate(tripOutside):
        html = trip_out['body']
        
        try:
            if (type(html) != list):
                tripOutside[i]['fix'] =  0
                text = html2text.html2text(html)
                text_pre = []
                for sentence in re.split('\[]|\n',text):
                    if (sentence != ''):
                        if('jpg' in sentence or 'gif' in sentence):
                            continue
                        else:
                            text_pre .append(sentence)
                tripOutside[i]['body'] =  text_pre
                tripOutside[i]['fix'] =  1
        except:
            print('Error',i)
            err_out.append(i)
    
    return tripInside, tripOutside,err_in,err_out

In [72]:
tripInside, tripOutside = get_text_forTrip(data,tripInsideIndex,tripOutsideIndex)

Error 692
Error 2617
Error 4837
Error 8554
Error 9327
Error 9330
Error 9342
Error 9377
Error 17863
Error 20694
Error 29213
Error 31953
Error 709
Error 1282
Error 2403
Error 3073
Error 3694
Error 13767
Error 14453
Error 15857
Error 22228
Error 23960
Error 28588
Error 34722
Error 35261
Error 38259
Error 39118
Error 43606
Error 45676
Error 47269
Error 47521


In [3]:
# with open('tripInside.pkl','wb') as f:
#     pickle.dump(tripInside,f)
# with open('tripOutide.pkl','wb') as f:
#     pickle.dump(tripOutside,f)
with open('tripInside.pkl','rb') as f:
    tripInside = pickle.load(f)
with open('tripOutide.pkl','rb') as f:
    tripOutide = pickle.load(f)

In [4]:
tripInside[1]

{'url': 'http://ub874001.pixnet.net/blog/post/457012652',
 'site_category': '國內旅遊',
 'custom_category': '中部旅遊',
 'title': ' 【台中好好玩PART 2】2017台中最新景點走拍/IG最熱門打卡五大景點/台中旅遊景點推薦',
 'tags': ['台中旅遊景點推薦', '台中二日遊行程', '台中必遊景點', '台中IG打卡景點', '創意時尚旅店', '台中住宿'],
 'body': ['台中洐然成為IG最佳代言人，打開IG一系列都是台中的美拍打卡景點，從2016年我是塔拉朵、好好小館、好好聚落、九天森林、勤美術館、草悟道、花匠花牆、台中歌劇院、到2017孵空間、柳川藍帶水岸、西洋博物館、勤美天地...等等無不是熱門打卡景點，趁著過年前全家安排一趟台中之旅，舉凡最新最夯最熱門的景點安排走走～',
  '[2018台中必遊景點【第六巿場/草悟道火箭扭蛋機泰迪熊奶茶公車/勤美入夢村】台中三大打卡新景點/全台最美的傳統巿場](http://ub874001.pixnet.net/blog/post/461373389-2018%e5%8f%b0%e4%b8%ad%e5%bf%85%e9%81%8a%e6%99%af%e9%bb%9e%e3%80%90%e7%ac%ac%e5%85%ad%e5%b7%bf%e5%a0%b4-%e8%8d%89%e6%82%9f%e9%81%93%e7%81%ab%e7%ae%ad%e6%89%ad%e8%9b%8b',
  '"2018台中必遊景點【第六巿場/草悟道火箭扭蛋機泰迪熊奶茶公車/勤美入夢村】台中三大打卡新景點/全台最美的傳統巿場")',
  '這次因為到勤美天地，就順道安排台中美食古早飯堂及花匠花牆',
  '沒想到花匠老闆是位日本帥哥，不儘飲料為現打果汁，加了優酪乳的飲品感覺好特別喔',
  '重點是很好喝，顏色鮮豔的漸層果汁加上店家特色花牆，呈現最美的一張圖～',
  '[!',
  '[!',
  '台中美拍新景點二日遊行程如下：',
  '第一天:桃園出發--垂坤伴手禮-高美濕地-台中好好小館-古早飯堂午餐-我是塔拉朵-勤美草悟道-花匠花牆-台中勤美術館-台中國家歌劇院-',
  '新天地西洋博物館

In [133]:
def get_tags(trip):
    tags_user = []
    tags_all = []
    for i,t in enumerate(trip):
        tag = t['tags']
        tag_pre = []
        for ta in tag:
            ta_p = ta.split('#')
            ta_p = [g.strip() for g in ta_p if g.strip() != ''] # split #
            ta_p = [re.sub(r'[^\w\s]','',g).strip() for g in ta_p if re.sub(r'[^\w\s]','',g).strip() != ''] # filter punctuation
            if(ta_p != ''):
                tag_pre.extend(ta_p)
        tag_pre = np.array(tag_pre)
        tag_pre = np.unique(tag_pre)
        
        tags_user.append(tag_pre)
        tags_all.extend(tag_pre)
    tags_user = np.array(tags_user)
    tags_all = np.array(tags_all)
    return tags_user, tags_all

In [134]:
tagsInsideUser, tagsInsideAll = get_tags(tripInside)
tagsInsideUnique = np.unique(tagsInsideAll)

In [135]:
tagsInsideUser

array([array(['嘉義', '奮起湖', '日出', '沼平車站', '火車', '神木車站', '租車', '螢火蟲', '阿里山'],
      dtype='<U4'),
       array(['創意時尚旅店', '台中IG打卡景點', '台中二日遊行程', '台中住宿', '台中必遊景點', '台中旅遊景點推薦'],
      dtype='<U8'),
       array(['下午茶', '台北火車站', '吃到飽', '君品', '精緻', '美食', '豪華', '食記'], dtype='<U5'),
       ...,
       array(['后里中社觀光花市', '泰安國小', '泰安火車站', '火車站', '烤肉', '鬱金香花海'], dtype='<U8'),
       array(['信義親子會館', '台北101', '台北景點', '四四南村', '外拍', '眷村文化', '眷村文物館'],
      dtype='<U6'),
       array(['宜蘭市景點', '宜蘭設治紀念館'], dtype='<U7')], dtype=object)

In [136]:
tagsInsideUnique[-105]

'龜吼漁港怎麼去交通方式'

In [138]:
regions = ['台北','新北','桃園','台中','台南','高雄','基隆','新竹','嘉義','苗栗','彰化','南投','雲林','屏東','宜蘭','花蓮','台東','澎湖']

In [140]:
# Save
with open('tagsInsideUnique.pkl','wb') as f:
    pickle.dump(tagsInsideUnique,f)
with open('tagsInsideUser.pkl','wb') as f:
    pickle.dump(tagsInsideUser,f)
with open('regions.pkl','wb') as f:
    pickle.dump(regions,f)

# # Load
# with open('tagsInsideUnique.pkl','rb') as f:
#     tagsInsideUnique = pickle.load(f)
# with open('tagsInsideUser.pkl','rb') as f:
#     tagsInsideUser = pickle.load(f)
# with open('regions.pkl','rb') as f:
#     regions = pickle.load(f)

In [139]:
len(regions)

18

### Government attraction style

In [11]:
attraction_json = requests.get('https://gis.taiwan.net.tw/XMLReleaseALL_public/scenic_spot_C_f.json')

In [12]:
attraction_dict_list = json.loads(attraction_json.text)['XML_Head']['Infos']['Info']

In [13]:
orgclass = []
for a in attraction_dict_list:
    if(a['Orgclass']):
        orgclass.append(a['Orgclass'])
orgclass = np.array(orgclass)

In [14]:
np.unique(orgclass)

array(['休閒景點', '休閒景點、文化景點', '休閒景點、特色展館', '公園遊憩', '其他', '古蹟類', '單車漫遊',
       '寺廟參訪', '展覽博物', '工藝創作', '廟宇古蹟', '廟宇類', '文化景點', '文化類', '文化類',
       '海岸風情', '溫泉類', '濱海風光', '特色展館', '特色展館、文化景點', '特色展館、觀光工廠', '生態景觀',
       '生態景觀、休閒景點、自行車道', '生態類', '綠野森呼吸', '縱情花田', '老街巡禮', '自然風景類', '藝術類',
       '親子同遊', '觀光工廠', '遊憩類', '鐵道懷舊'], dtype='<U14')

In [15]:
def get_attraction_name(attraction_dict_list):
    names = []
    key_words = []
    for a in attraction_dict_list:
        names.append(a['Name'])
        key_words.append(a['Keyword'])
    names = np.array(names)
    key_words = np.array(key_words)
    
    return names,key_words

In [16]:
names,key_words = get_attraction_name(attraction_dict_list)

In [17]:
names.shape

(5158,)

In [18]:
def get_overlap(names,tagsInsideAll):
    overlap_attr = []
    uniquetag = np.unique(tagsInsideAll)
    for n in names:
        if n in uniquetag:
            overlap_attr.append(n)
    return overlap_attr

In [19]:
overlap_attr = get_overlap(names,tagsInsideAll)

In [20]:
len(overlap_attr)

1911

In [21]:
print(overlap_attr)

['石梯坪', '長虹橋', '北回歸線', '金樽', '加路蘭', '都蘭', '阿美族民俗中心', '紫坪', '帆船鼻大草原', '富山漁業資源保育區', '花蓮遊客中心', '都歷遊客中心', '月光小棧', '比西里岸', '金剛大道', '鹽寮', '目斗嶼', '吉貝嶼', '姑婆嶼', '險礁嶼', '員貝嶼', '眷村文化園區', '中央街', '中屯風力園區', '通梁古榕', '澎湖跨海大橋', '小門嶼', '西嶼西臺', '漁翁島燈塔', '網垵口沙灘', '雙心石滬', '大獅風景區', '小台灣', '望夫石', '七美人塚', '西嶼東臺軍事史蹟園區', '蛇頭山', '鳥嶼', '龍埕', '萬軍井', '施公祠', '順承門', '南海遊客中心', '北海遊客中心', '牛心山', '新光兆豐休閒農場', '馬太鞍休閒農業區', '油菜花海', '池上飯包文化故事館', '布農部落休閒農場', '環灣自行車道', '大鵬灣遊客中心', '大鵬灣國際休閒特區', '東隆宮', '鵬灣跨海大橋', '潟湖', '蚵殼島', '崎峰濕地', '烏鬼洞', '蛤板灣沙灘', '白燈塔', '美人洞', '山豬溝', '厚石裙礁', '碧雲寺', '三隆宮', '中澳沙灘', '恆春古城門', '七孔瀑布', '港口吊橋', '九棚大沙漠', '鵝鑾鼻', '白砂灣', '瓊麻工業歷史展示區', '小灣', '落日灣', '八八坑道', '牛角聚落', '枕戈待旦紀念公園', '民俗文物館', '津沙聚落', '大漢據點', '馬祖天后宮', '印地安人頭岩', '烈女義坑', '一線天', '中柱島', '感恩亭', '國之北疆', '羅漢坪', '后澳', '大浦聚落', '大埔石刻', '福正聚落', '戰爭和平紀念公園', '后澳村', '龜島', '壁山', '大坵', '白沙港', '坂里沙灘', '猫囒山步道', '雙龍瀑布', '龍鳳宮', '頭社盆地', '親手窯', '潭南國小', '慈恩塔步道', '敲敲木工房', '慈恩塔', '集集綠色隧道', '涵碧步道', '梅荷園', '草湳溼地', '紙教堂', '紙匠工房', '蝴蝶園', '金鋼基地', '泥炭土活盆地', '武昌宮', '松柏崙步道', '明潭電廠'

### TripAdvisior

In [210]:
headers = {'x-requested-with': 'XMLHttpRequest'}

In [211]:
url = 'https://www.tripadvisor.com.tw/Search?geo=297910&searchNearby=&pid=3825&redirect=&startTime=1559624037161&uiOrigin=MASTHEAD&q=%E5%8F%B0%E4%B8%AD&supportedSearchTypes=find_near_stand_alone_query&enableNearPage=true&returnTo=https:__2F____2F__www__2E__tripadvisor__2E__com__2E__tw__2F__Attraction__5F__Review__2D__g13806822__2D__d1047955__2D__Reviews__2D__Cingjing__5F__Veterans__5F__Farm__2D__Ren__5F__ai__5F__Township__5F__Nantou__2E__html&searchSessionId=405D6B4ECF12B0C7A7CD56771442AFB81559624014157ssid&social_typeahead_2018_feature=true&sid=405D6B4ECF12B0C7A7CD56771442AFB81559624041098&ssrc=A&rf=4'
res = requests.get(url,auth=('user', 'pass'),headers=headers)
doc = pq(res.text)

In [220]:
doc('.result-title').text()

'高鐵台中站 台中站 逢甲夜市 臺中國家歌劇院 台中公園 台中爵士音樂節 台中時空漫步 望高寮夜景公園 高美濕地 一中街商圈 新光三越 台中中港店 道禾六藝文化館-台中刑務所演武場 台中大遠百 彩虹眷村 東海大學 文化部文化資産園区 台中市政府大樓 台中都會公園 草悟道 第二市場 國立自然科學博物館 孔廟忠烈祠 大魯閣新時代 誠品綠園道 美術園道 廣三崇光百貨（台中） 台中洲際棒球場 大甲鎮瀾宮 勤美術館 柳川藍帶水岸(親水河岸)'

In [258]:
for i,d in enumerate(doc(".styleguide a").items()):
    print(i,d.attr('href'))

0 None
1 javascript:void(0);
2 javascript:void(0);
3 javascript:void(0);
4 javascript:void(0);
5 javascript:void(0);
6 javascript:void(0);
7 javascript:void(0);
8 javascript:void(0);


In [259]:
from opencc import OpenCC

In [266]:
cc = OpenCC('s2twp')
converted = cc.convert(doc('.result-title').text())

In [267]:
converted

'高鐵臺中站 臺中站 逢甲夜市 臺中國家歌劇院 臺中公園 臺中爵士音樂節 臺中時空漫步 望高寮夜景公園 高美濕地 一中街商圈 新光三越 臺中中港店 道禾六藝文化館-臺中刑務所演武場 臺中大遠百 彩虹眷村 東海大學 文化部文化資産園區 臺中市政府大樓 臺中都會公園 草悟道 第二市場 國立自然科學博物館 孔廟忠烈祠 大魯閣新時代 誠品綠園道 美術園道 廣三崇光百貨（臺中） 臺中洲際棒球場 大甲鎮瀾宮 勤美術館 柳川藍帶水岸(親水河岸)'